http://rebound.readthedocs.io/en/latest/ipython/Horizons.html

In [ ]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound

# Saturn

In [2]:
filename = "saturn.bin"
#try:
    #sim = rebound.Simulation.from_file("../binaries/"+filename)
#except:
date = "2017-06-04 05:00" # barycentric dynamical time (sounded good but DOESN't work anymore, P<0, positions all very close)
#date = "2017-09-15 12:08" # barycentric dynamical time
#date = "2017-06-17 02:00" # barycentric dynamical time (24*4=96 orbits of mimas before plunge)
#date = "2017-09-12 05:37" # barycentric dynamical time DOESN't work, P<0

#grand finale: Sept. 15: Final signal received on Earth at 12:08 p.m. UTC (5:08 a.m. PDT)
#Start of final orbit Apoapse Sep 12 5:37 a.m.UTC 10:37 p.m. PDT
sim = rebound.Simulation()
sim.add("Saturn", date=date)  #sim.add("NAME=Saturn", date=date) 
sim.add("Mimas")
sim.add("Enceladus")
sim.add("Tethys")
sim.add("Dione")
sim.add("Rhea")
sim.add("Titan")
sim.add("Iapetus")
print(sim.particles[-1].P)
sim.move_to_com()

ps = sim.particles
sim2 = rebound.Simulation()
sim2.add(m=ps[0].m)
for p in ps[1:]:
    sim2.add(m=p.m, a=p.a, e=p.e, pomega=p.pomega, theta=p.theta)
sim2.move_to_com()
sim2.save("../binaries/"+filename)

Searching NASA Horizons for 'Saturn'... Found: Saturn Barycenter (6).
Searching NASA Horizons for 'Mimas'... Found: Mimas (601).
Searching NASA Horizons for 'Enceladus'... Found: Enceladus (602).
Searching NASA Horizons for 'Tethys'... Found: Tethys (603).
Searching NASA Horizons for 'Dione'... Found: Dione (604).
Searching NASA Horizons for 'Rhea'... Found: Rhea (605).
Searching NASA Horizons for 'Titan'... Found: Titan (606).
Searching NASA Horizons for 'Iapetus'... Found: Iapetus (608).
1.3651720005602326


# Arbitrary System

In [8]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
import random
filename = "newsystem.bin"

sim = rebound.Simulation()
sim.add(m=1.)  #1 Solar mass
a1=1.
sim.add(m=1e-6, a=a1, e=0., inc=0., theta=2.*np.pi*np.random.rand())
a2=a1*(3./2.)**(-2./3)  #semi-major axis of a perfect 5th above planet 1
sim.add(m=1e-6, a=a2, theta=2.*np.pi*np.random.rand())
a3=a2*(4./3.)**(-2./3)  #semi-major axis 4th above planet 2 (=a1*2**(-2./3))
sim.add(m=1e-6, a=a3, theta=2.*np.pi*np.random.rand())
'''
a4=a3*(4./3.)**(-2./3)  #semi-major axis 4th above planet 2 (=a1*2**(-2./3))
sim.add(m=1e-6, a=a4, theta=2.*np.pi*np.random.rand())
a5=a4*(4./3.)**(-2./3)  #semi-major axis 4th above planet 2 (=a1*2**(-2./3))
sim.add(m=1e-6, a=a5, theta=2.*np.pi*np.random.rand())
a6=a5*(4./3.)**(-2./3)  #semi-major axis 4th above planet 2 (=a1*2**(-2./3))
sim.add(m=1e-6, a=a6, theta=2.*np.pi*np.random.rand())
a7=a6*(4./3.)**(-2./3)  #semi-major axis 4th above planet 2 (=a1*2**(-2./3))
sim.add(m=1e-6, a=a7, theta=2.*np.pi*np.random.rand())
'''
sim.status()

sim.move_to_com()
sim.save("../binaries/"+filename)

---------------------------------
REBOUND version:     	3.5.2
REBOUND built on:    	Jun 18 2017 23:59:37
Number of particles: 	8
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=1e-06 x=-0.8716730871507681 y=0.49008777697168643 z=0.0 vx=-0.49008802201551366 vy=-0.8716735229872028 vz=0.0>
<rebound.Particle object, m=1e-06 x=-0.730420639335784 y=0.22107500127562923 z=0.0 vx=-0.3316125884816464 vy=-1.0956316187971078 vz=0.0>
<rebound.Particle object, m=1e-06 x=-0.5483225950433971 y=0.3101528539545719 z=0.0 vx=-0.6203060377411296 vy=-1.0966455981687016 vz=0.0>
<rebound.Particle object, m=1e-06 x=0.38604180475603983 y=-0.34841232388606197 z=0.0 vx=0.929102670063341 vy=1.0294495420665808 vz=0.0>
<rebound.Particle object, m=1e-06 x=-0.22664449864592479 y=-0.36455926768404756 z=0.0 vx=1.296215849705634 vy=-0

# Janus and Epimetheus

In [11]:

import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
filename = "janus.bin"
#try:
    #sim = rebound.Simulation.from_file("../binaries/"+filename)
#except:
date = "2017-06-04 00:57"
sim = rebound.Simulation()
sim.add("Saturn", date=date)
sim.add("Epimetheus", date=date)
#print('Epimetheus a=',sim.particles[-1].a, 'P=',sim.particles[-1].P)
sim.add("Janus", date=date)
#print('Janus a=',sim.particles[-1].a, 'P=',sim.particles[-1].P)
sim.move_to_com()

ps = sim.particles
sim2 = rebound.Simulation()
sim2.add(m=ps[0].m)
for p in ps[1:]:
    sim2.add(m=p.m, a=p.a, e=p.e, pomega=p.pomega, theta=p.theta)
    
#add ring of particles at Janus's 2:1 resonance
a_Janus=sim.particles[-1].a
N_ring=5
for i in range(N_ring):
    sim2.add(m=0.,a=a_Janus*2**(-2./3),theta=2.*np.pi/N_ring*i)
    
sim2.move_to_com()
sim2.save("../binaries/"+filename)

Searching NASA Horizons for 'Saturn'... Found: Saturn Barycenter (6).
Searching NASA Horizons for 'Epimetheus'... Found: Epimetheus (611).
Searching NASA Horizons for 'Janus'... Found: Janus (610).


# Janus Resonances

In [68]:

import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
filename = "janus_all.bin"
#try:
    #sim = rebound.Simulation.from_file("../binaries/"+filename)
#except:
date = "2017-06-04 00:57"
sim = rebound.Simulation()
sim.add("Saturn", date=date)
#sim.add("Epimetheus", date=date)
#print('Epimetheus a=',sim.particles[-1].a, 'P=',sim.particles[-1].P)
sim.add("Janus", date=date)
#print('Janus a=',sim.particles[-1].a, 'P=',sim.particles[-1].P)
sim.move_to_com()

ps = sim.particles
sim2 = rebound.Simulation()
sim2.add(m=ps[0].m)
for p in ps[1:]:
    sim2.add(m=p.m, a=p.a, e=p.e, pomega=p.pomega, theta=p.theta)
    
#add ring of particles at Janus's 2:1 resonance
a_Janus=sim.particles[-1].a
sim.remove(1)

for m in [7,6,5,4,3,2]:
    N_ring=5
    for i in range(N_ring):
        sim2.add(m=0.,a=a_Janus*(m/(m-1))**(-2./3),theta=2.*np.pi/N_ring*i)
        
sim.add("Epimetheus", date=date)
sim.add("Janus", date=date)
sim2.move_to_com()
sim2.save("../binaries/"+filename)

Searching NASA Horizons for 'Saturn'... Found: Saturn Barycenter (6).
Searching NASA Horizons for 'Janus'... Found: Janus (610).
Searching NASA Horizons for 'Epimetheus'... Found: Epimetheus (611).
Searching NASA Horizons for 'Janus'... Found: Janus (610).


# Solar System

In [ ]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
filename = "solarsystem.bin"
#try:
    #sim = rebound.Simulation.from_file("../binaries/"+filename)
#except:
date = "1983-02-17 02:00"
date = "2017-05-10 05:00"

sim = rebound.Simulation()
sim.add("Sun", date=date)
sim.add("Mercury", date=date)
sim.add("Venus", date=date)
sim.add("Earth", date=date)
sim.add("Mars", date=date)
sim.add("Jupiter", date=date)
sim.add("Saturn", date=date)
sim.add("Uranus", date=date)
sim.add("Neptune", date=date)
sim.move_to_com()
sim.save("../binaries/"+filename)

In [4]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
filename = "solarsystemI.bin"
#try:
    #sim = rebound.Simulation.from_file("../binaries/"+filename)
#except:
date = "1983-02-17 02:00"
date = "2017-05-10 05:00"

sim = rebound.Simulation()
sim.add("Sun", date=date)
sim.add("Mercury", date=date)
sim.add("Venus", date=date)
sim.add("Earth", date=date)
sim.add("Mars", date=date)
sim.move_to_com()
sim.save("../binaries/"+filename)

Searching NASA Horizons for 'Sun'... Found: Sun (10).
Searching NASA Horizons for 'Mercury'... Found: Mercury Barycenter (199).
Searching NASA Horizons for 'Venus'... Found: Venus Barycenter (299).
Searching NASA Horizons for 'Earth'... Found: Earth-Moon Barycenter (3).
Searching NASA Horizons for 'Mars'... Found: Mars Barycenter (4).


# Asteroid Belt

In [70]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
import pandas as pd
filename = "asteroidbelt_m.bin"
#try:
    #sim = rebound.Simulation.from_file("../binaries/"+filename)
#except:

data='asteroids_100_names.csv'   #choose minium size to include, 10 or 100 km
asteroids=pd.read_csv(data, sep=',',header=None)
header=asteroids.values[0]
print(header)
print(asteroids.values[1])

ast_IDs=[]
ast_names=[]
ast_as=[]
ast_is=[]
ast_oms=[]
ast_ws=[]
ast_es=[]

a_max = 5.
for i in range(1,len(asteroids.values)):
    if float(asteroids.values[i][2])<a_max:
        ast_names.append(asteroids.values[i][0])
        ast_as.append(float(asteroids.values[i][2]))
        ast_es.append(float(asteroids.values[i][1]))
        ast_is.append(float(asteroids.values[i][3])*np.pi/180)
        ast_oms.append(float(asteroids.values[i][4])*np.pi/180)
        ast_ws.append(float(asteroids.values[i][5])*np.pi/180)
#print(ast_IDs)
        
date = "2017-05-10 05:00"

sim = rebound.Simulation()
sim.add("Sun", date=date)
sim.add("Mars", date=date)
for i in range(len(ast_as)):
    sim.add("NAME="+ast_names[i])
    #sim.add(a=ast_as[i],e=ast_es[i],inc=ast_is[i],Omega=ast_oms[i],omega=ast_ws[i])

sim.move_to_com()
sim.save("../binaries/"+filename)


['name' 'e' 'a' 'i' 'om' 'w' 'per']
['Flora' '0.157028745' '2.201405117' '5.886535185' '110.9009246'
 '285.3102149' '1193.021813']
Searching NASA Horizons for 'Sun'... Found: Sun (10).
Searching NASA Horizons for 'Mars'... Found: Mars Barycenter (4).
Searching NASA Horizons for 'NAME=Flora'... Found: 8 Flora.
Searching NASA Horizons for 'NAME=Harmonia'... Found: 40 Harmonia.
Searching NASA Horizons for 'NAME=Melpomene'... Found: 18 Melpomene.
Searching NASA Horizons for 'NAME=Zelinda'... Found: 654 Zelinda (1908 BM).
Searching NASA Horizons for 'NAME=Victoria'... Found: 12 Victoria.
Searching NASA Horizons for 'NAME=Vesta'... Found: 4 Vesta.
Searching NASA Horizons for 'NAME=Nemausa'... Found: 51 Nemausa.
Searching NASA Horizons for 'NAME=Artemis'... Found: 105 Artemis.
Searching NASA Horizons for 'NAME=Athamantis'... Found: 230 Athamantis.
Searching NASA Horizons for 'NAME=Iris'... Found: 7 Iris.
Searching NASA Horizons for 'NAME=Metis'... Found: 9 Metis.
Searching NASA Horizons for '

Searching NASA Horizons for 'NAME=Winchester'... Found: 747 Winchester (1913 QZ).
Searching NASA Horizons for 'NAME=Marion'... Found: 506 Marion (1903 LN).
Searching NASA Horizons for 'NAME=Isolda'... Found: 211 Isolda.
Searching NASA Horizons for 'NAME=Armida'... Found: 514 Armida (1903 MB).
Searching NASA Horizons for 'NAME=Germania'... Found: 241 Germania.
Searching NASA Horizons for 'NAME=Aegle'... Found: 96 Aegle.
Searching NASA Horizons for 'NAME=Emma'... Found: 283 Emma.
Searching NASA Horizons for 'NAME=Interamnia'... Found: 704 Interamnia (1910 KU).
Searching NASA Horizons for 'NAME=Patientia'... Found: 451 Patientia (1899 EY).
Searching NASA Horizons for 'NAME=Arethusa'... Found: 95 Arethusa.
Searching NASA Horizons for 'NAME=Gerlinde'... Found: 663 Gerlinde (1908 DG).
Searching NASA Horizons for 'NAME=Diotima'... Found: 423 Diotima (1896 DB).
Searching NASA Horizons for 'NAME=Marianna'... Found: 602 Marianna (1906 TE).
Searching NASA Horizons for 'NAME=Adorea'... Found: 268 

In [62]:
#a0000006 130.31 C3 -6.7 cents      6 Hebe
#a0000046 122.61 B2 -12.1 cents     46 Hestia
#a0000146 109.83 A2 -2.6 cents    146 Lucina
#a0000266 104.74 G#2 +15.2 cents    266 Aline
#a0000776 98.06 G2 +1.0 cents    776 Berbericia (1914 TY)
#a0000488 87.22 F2 -1.7 cents    488 Kreusa (1902 JG)
#a0000334 64.05 C2 -36.4 cents    334 Chicago (1892 L) 198.77
ast_notes=['Hebe','Hestia','Lucina','Aline','Berbericia','Kreusa','Chicago']
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
import pandas as pd
filename = "asteroidnotes.bin"
#try:
    #sim = rebound.Simulation.from_file("../binaries/"+filename)
#except:

data='asteroids_100_names.csv'   #choose minium size to include, 10 or 100 km
asteroids=pd.read_csv(data, sep=',',header=None)
header=asteroids.values[0]
print(header)
print(asteroids.values[1])

ast_IDs=[]
ast_names=[]
ast_as=[]
ast_is=[]
ast_oms=[]
ast_ws=[]
ast_es=[]

for i in range(1,len(asteroids.values)):
    if asteroids.values[i][0] in ast_notes:
        ast_names.append(asteroids.values[i][0])
        ast_as.append(float(asteroids.values[i][2]))
        ast_es.append(float(asteroids.values[i][1]))
        ast_is.append(float(asteroids.values[i][3])*np.pi/180)
        ast_oms.append(float(asteroids.values[i][4])*np.pi/180)
        ast_ws.append(float(asteroids.values[i][5])*np.pi/180)
#print(ast_IDs)
        
date = "2017-05-10 05:00"

sim = rebound.Simulation()
sim.add("Sun", date=date)

for i in range(len(ast_as)):
    sim.add("NAME="+ast_names[i])

sim.move_to_com()
sim.save("../binaries/"+filename)

['name' 'e' 'a' 'i' 'om' 'w' 'per']
['Flora' '0.157028745' '2.201405117' '5.886535185' '110.9009246'
 '285.3102149' '1193.021813']
Searching NASA Horizons for 'Sun'... Found: Sun (10).
Searching NASA Horizons for 'NAME=Hebe'... Found: 6 Hebe.
Searching NASA Horizons for 'NAME=Hestia'... Found: 46 Hestia.
Searching NASA Horizons for 'NAME=Lucina'... Found: 146 Lucina.
Searching NASA Horizons for 'NAME=Aline'... Found: 266 Aline.
Searching NASA Horizons for 'NAME=Berbericia'... Found: 776 Berbericia (1914 TY).
Searching NASA Horizons for 'NAME=Kreusa'... Found: 488 Kreusa (1902 JG).
Searching NASA Horizons for 'NAME=Chicago'... Found: 334 Chicago (1892 L).


# Calculate Pitches

In [66]:
notenames = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
A4 = 440
C0 = A4*pow(2, -4.75)
C4=C0*pow(2, 4)

def pitch(freq):
    h = int(round(12*np.log2(freq/C0)))
    octave = h // 12
    n = h % 12
    f0=C0*2**(h/12.)
    cent = 1200*np.log2(freq/f0)
    #return name[n] + str(octave) + ' + '+str(round(cent,1)) +' cents'
    return '{0} {1:+.1f} cents'.format(notenames[n]+str(octave),cent)

filename = "../binaries/saturn.bin"
sim = rebound.Simulation.from_file(filename)

basefreq=C4 #or 1/timescale*2**octave
basefreq=103.068333727
baseperiod=sim.particles[1].P
freqs=[basefreq*baseperiod/sim.particles[i].P for i in range(1,sim.N)]

basefreq=130.8
freqs=[basefreq,basefreq*7./6,basefreq*6./5,basefreq*5./4,basefreq*4./3]

print('\nNotes and offsets:')
for i in range(1,sim.N):
    print('Planet '+str(i)+' : '+ pitch(freqs[i-1]),' = ',np.round(freqs[i-1],2), 'Hz')


Notes and offsets:
Planet 1 : C3 -0.2 cents  =  130.8 Hz
Planet 2 : D#3 -33.3 cents  =  152.6 Hz
Planet 3 : D#3 +15.5 cents  =  156.96 Hz
Planet 4 : E3 -13.9 cents  =  163.5 Hz
Planet 5 : F3 -2.1 cents  =  174.4 Hz


IndexError: list index out of range

In [16]:
print(freqs)

[103.068333727, 58.666437667350635, 38.85028963839458, 24.839169084682965, 13.600286058304766, 1.8832410884449529, -0.1726740998045721]


In [ ]:
filename = "../binaries/solarsystem.bin"
sim = rebound.Simulation.from_file(filename)
sim.t = 0
ss.rescale_time(sim, sim.particles[3].P) #Earth

In [6]:
transits = ss.EventRecorder(sim, lambda sim, i: sim.particles[i].y, targets=[])
conjunctions = ss.EventRecorder(sim, lambda sim, i: np.sin(sim.particles[i].theta - sim.particles[i+1].theta), targets=[])

We start by staggering in the planet transits, 4 outer planet orbits at a time, from the outside inward, by changing `transits.targets`.

In [7]:
planets = list(range(sim.N))
for i in range(1,8):
    ps = planets[-i:]
    transits.targets = ps
    print(transits.targets)
    sim.integrate(tmax=sim.t+4)

[7]
[6, 7]
[5, 6, 7]
[4, 5, 6, 7]
[3, 4, 5, 6, 7]
[2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]


We now stagger in conjunctions starting from the outermost pair, leaving in all the planets for the transits:

In [8]:
planets = list(range(sim.N-1))
for i in range(1,7):
    ps = planets[-i:]
    conjunctions.targets = ps
    print(conjunctions.targets)
    sim.integrate(tmax=sim.t+4)

[6]
[5, 6]
[4, 5, 6]
[3, 4, 5, 6]
[2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 6]


In [ ]:
planets = list(range(sim.N))
ps = planets
for i in range(100):
    transits.targets = ps
    print(transits.targets)
    sim.integrate(tmax=sim.t+4)

# Making the MIDI

We need to set a tempo in beats per minute, where 1 beat corresponds to one time unit in the simulation. Here we set bpm=30, which means that there are 2 seconds per simulation time unit = one outer planet orbit. We then make the MIDI file and set the tempo:

In [9]:
from midiutil import MIDIFile

bpm = 30
midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=0, tempo=bpm) 

Now we need to decide on MIDI notes for the transits and conjunctions. We use the `calc_midi_notes` function to scale all notes relative to the outermost (ref_ID=-1) particle, and assign to that planet a MIDI note of 48=C4. See e.g.  http://subsynth.sourceforge.net/midinote2freq.html for a list of MIDI notes. We then manually assign notes to the conjunctions.

Finally, we step through the list of transits and conjunctions, and add the corresponding note for each of them, depending on the transiting planet or conjunction pair, at the time of the event. One can also set the note duration and velocity (volume). Here we add planet transits onto separate channels, and the conjunctions all into one channel above the others.

In [10]:
transit_notes = ss.calc_midi_notes(sim.particles, ref_note=48, ref_ID=-1)
conjunction_notes = [0, 33, 35, 20, 18, 14, 12]

for transit in transits.events:
    midifile.addNote(track=0, channel=transit['target'], pitch=transit_notes[transit['target']], time=transit['time'], duration=1, volume=100)
for conjunction in conjunctions.events:
    midifile.addNote(track=0, channel=sim.N, pitch=conjunction_notes[conjunction['target']], time=conjunction['time'], duration=1, volume=100)

Finally, we write the file:

In [11]:
with open("./trappist.mid", "wb") as f:
    midifile.writeFile(f)